In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 开始使用Vertex ML Metadata

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/get_started_with_vertex_ml_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/get_started_with_vertex_ml_metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ml_metadata/get_started_with_vertex_ml_metadata.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>            
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用 Vertex ML Metadata。

了解更多关于[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata)。

### 目标

在本教程中，您将学习如何使用 `Vertex ML Metadata`。

此教程使用以下 Google Cloud ML 服务：

- `Vertex ML Metadata`
- `Vertex AI Pipelines`

执行的步骤包括：

- 创建一个 `Metadatastore` 资源。
- 创建（记录）/列出一个带有工件和元数据的 `Artifact`。
- 创建（记录）/列出一个 `Execution`。
- 创建（记录）/列出一个 `Context`。
- 将 `Artifact` 添加到 `Execution` 作为事件。
- 将 `Execution` 和 `Artifact` 添加到 `Context` 中。
- 删除 `Artifact`、`Execution` 和 `Context`。
- 创建并运行一个 `Vertex AI Pipeline` 机器学习工作流来训练和部署一个 scikit-learn 模型。
    - 创建自定义管道组件来生成工件和元数据。
    - 比较 Vertex AI Pipelines 运行。
    - 跟踪管道生成的工件的谱系。
    - 查询您的管道运行元数据。

### 数据集

这个教程使用的数据集是UCI机器学习中的 ['干豆数据集'](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset), 参考：KOKLU, M. and OZKAN, I.A., (2020), "基于计算机视觉和机器学习技术的干豆多分类", 在《计算与电子农业》（Computers and Electronics in Agriculture）杂志上发表的文章，174卷，105507号。[DOI](https://doi.org/10.1016/j.compag.2020.105507)。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud 存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud 存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装执行笔记本所需的软件包。

In [ ]:
import os

! pip3 install --upgrade google-cloud-aiplatform[tensorboard] \
                         google-cloud-pipeline-components  --quiet

只有Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，这样它才能找到这些包。

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`。
- 运行 `gcloud projects list`。
- 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

地区

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 身份验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

1. Vertex AI 工作台
* 无需操作，您已经通过身份验证。

本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
IS_COLAB = False
# from google.colab import auth
# auth.authenticate_user()
# IS_COLAB = True

4. 服务帐户或其他
*请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务帐户授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时 才能运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

服务账户

**如果您不知道您的服务账户**，请尝试通过执行下面的第二个单元格中的`gcloud`命令来获取您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置Vertex AI Pipelines的服务账号访问权限

运行以下命令，为您的服务账号授予对在上一步创建的存储桶中的流水线工件进行读取和写入的访问权限-- 您只需要针对每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

设置变量

接下来，设置一些在本教程中使用的变量。

导入Vertex AI SDK

将Vertex AI SDK导入到您的Python环境中。

In [ ]:
from datetime import datetime

import google.cloud.aiplatform_v1beta1 as aip_beta
from google.cloud import aiplatform

#### Vertex AI常量

为Vertex AI设置以下常量：

- `API_ENDPOINT`：用于`ML Metadata`服务的Vertex AI API服务端点。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

设置客户端

Vertex 作为一个客户端/服务器模型。在您一侧（Python脚本），您将创建一个客户端，向Vertex AI服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 元数据服务用于创建记录、搜索和分析工件和元数据。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_metadata_client():
    client = aip_beta.MetadataServiceClient(client_options=client_options)
    return client


clients = {}
clients["metadata"] = create_metadata_client()

for client in clients.items():
    print(client)

## 介绍 Vertex AI Metadata

`Vertex ML Metadata` 服务为您提供了记录、搜索和分析您的 ML 工作流产生的工件和相应元数据的能力。例如，在试验过程中，您可能希望记录模型工件的位置作为工件，并将训练超参数和评估指标作为相应的元数据。

该服务支持手动和自动记录 ML 元数据，后者在使用 Vertex AI Pipelines 时发生。

### 概念和组织

Vertex ML Metadata 将您的 ML 系统的元数据描述为图。

**工件**：工件是 ML 系统消费或产生的数据片段，如数据集、模型或日志。对于大型工件，如数据集或模型，工件记录包括数据存储的 URI。

**执行**：执行描述了 ML 系统工作流程中的单个步骤。

**事件**：执行可以依赖于工件作为输入或产生工件作为输出。事件描述了工件和执行之间的关系，以帮助您确定工件的继承关系。例如，会创建一个事件来记录数据集被执行使用，另一个事件记录该执行生成了一个模型。

**上下文**：上下文让您将工件和执行组合在一个可查询和类型化的类别中。

### ML 工件继承

Vertex ML Metadata 提供了理解您的机器学习系统性能变化的能力，以及分析您的 ML 工作流产生的元数据和其工件继承关系。工件的继承关系包括导致其创建的所有因素，以及由此工件传递的工件和元数据。

了解更多有关[介绍 Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) 的信息。

### 创建一个`MetadataStore`资源

每个项目可以拥有一个或多个`MetadataStore`资源。默认情况下，如果没有明确创建，每个项目都有一个默认的，其指定为：

    projects/<project_id>/locations/<region>/metadataStores/<name>

您可以使用`create_metadata_store()`方法创建一个`MetadataStore`资源，具有以下参数：

- `parent`：您项目中所有资源的完全限定子路径，即，projects/<project_id>/locations/<location>
- `metadata_store_id`：`MetadataStore`资源的名称。

In [ ]:
metadata_store = clients["metadata"].create_metadata_store(
    parent=PARENT, metadata_store_id="my-metadata-store-unique"
)

metadata_store_id = str(metadata_store.result())[7:-2]
print(metadata_store_id)

### 列出元数据模式

当您创建`Artifact`、`Execution`或`Context`资源时，您需要指定一个描述相应元数据的模式。这些模式必须在您的`Metadatastore`资源中预先注册。

您可以使用`list_metadata_schemas()`方法获取所有已注册的模式列表，包括默认和用户定义的，具体参数如下：

- `name`: `MetadataStore`资源的完全限定资源标识符。

了解更多关于[元数据系统模式](https://cloud.google.com/vertex-ai/docs/ml-metadata/system-schemas)。

In [ ]:
schemas = clients["metadata"].list_metadata_schemas(parent=metadata_store_id)

for schema in schemas:
    print(schema)

### 创建一个`Artifact`资源

您可以使用`create_artifact()`方法创建一个`Artifact`资源，需要使用以下参数:

- `parent`：指向`Metadatastore`资源的完全限定资源标识符。
- `artifact`：`Artifact`资源的定义
    - `display_name`：`Artifact`资源的人类可读名称。
    - `uri`：Artifact文件的统一资源标识符。如果没有实际的Artifact文件，则可以为空。
    - `labels`：要分配给`Artifact`资源的用户定义标签。
    - `schema_title`：描述元数据的模式的标题。
    - `metadata`：要与`Artifact`资源关联的元数据键/值对。
- `artifact_id`：（可选）用户定义的`Artifact`资源的短ID。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Artifact

artifact_item = Artifact(
    display_name="my_example_artifact",
    uri="my_url",
    labels={"my_label": "value"},
    schema_title="system.Artifact",
    metadata={"param": "value"},
)

artifact = clients["metadata"].create_artifact(
    parent=metadata_store_id,
    artifact=artifact_item,
    artifact_id="myartifactid",
)

print(artifact)

### 在 `Metadatastore` 中列出 `Artifact` 资源

您可以使用 `list_artifacts()` 方法列出所有 `Artifact` 资源，参数如下：

- `parent`: `MetadataStore` 资源的完全限定资源标识符。

In [ ]:
artifacts = clients["metadata"].list_artifacts(parent=metadata_store_id)

for _artifact in artifacts:
    print(_artifact)

### 创建一个`Execution`资源

您可以使用`create_execution()`方法创建一个`Execution`资源，需要以下参数：

- `parent`：`Metadatastore`资源的完全限定资源标识符。
- `execution`：
    - `display_name`：`Execution`资源的人类可读名称。
    - `schema_title`：描述元数据的模式标题。
    - `metadata`：要与`Execution`资源关联的元数据键值对。
- `execution_id`：（可选）用户定义的`Execution`资源的短ID。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Execution

execution = clients["metadata"].create_execution(
    parent=metadata_store_id,
    execution=Execution(
        display_name="my_execution",
        schema_title="system.CustomJobExecution",
        metadata={"value": "param"},
    ),
    execution_id="myexecutionid",
)

print(execution)

### 在`Metadatastore`中列出`Execution`资源

您可以使用`list_executions()`方法列出所有`Execution`资源，需要以下参数：

- `parent`：`MetadataStore`资源的完全限定资源标识符。

In [ ]:
executions = clients["metadata"].list_executions(parent=metadata_store_id)

for _execution in executions:
    print(_execution)

### 创建一个`Context`资源

您可以使用`create_context()`方法来创建一个`Context`资源，需要提供以下参数：

- `parent`：指向`Metadatastore`资源的完全限定资源标识符。
- `context`：
    - `display_name`：对`Execution`资源的可读名称。
    - `schema_title`：描述元数据的模式标题。
    - `labels`：用户定义的要分配给`Context`资源的标签。
    - `metadata`：要与`Execution`资源关联的元数据键值对。
- `context_id`：（可选）用户定义的`Context`资源的简短ID。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Context

context = clients["metadata"].create_context(
    parent=metadata_store_id,
    context=Context(
        display_name="my_context",
        labels=[{"my_label", "my_value"}],
        schema_title="system.Pipeline",
        metadata={"param": "value"},
    ),
    context_id="mycontextid",
)

print(context)

在`Metadatastore`中列出`Context`资源

您可以使用`list_contexts()`方法列出所有`Context`资源，并使用以下参数：

- `parent`：`MetadataStore`资源的完全合格的资源标识符。

In [ ]:
contexts = clients["metadata"].list_contexts(parent=metadata_store_id)

for _context in contexts:
    print(_context)

### 将事件添加到“Execution”资源

“Execution”资源由执行过程中发生的一系列事件组成。每个事件包括一个作为“Execution”资源的输入或输出的工件。

您可以使用`add_execution_events()`方法向“Execution”资源添加执行事件，参数如下：

- `execution`：`Execution`资源的完全限定资源标识符。
- `events`：构成执行过程的事件序列。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Event

clients["metadata"].add_execution_events(
    execution=execution.name,
    events=[
        Event(
            artifact=artifact.name,
            type_=Event.Type.INPUT,
            labels={"my_label": "my_value"},
        )
    ],
)

### 将工件和执行合并为上下文

上下文用于将“工件”资源和“执行”资源组合在一个可查询和类型化的类别下。上下文可用于表示元数据集。

您可以使用`add_context_artifacts_and_executions（）`方法将一组`Artifact`和`Execution`资源合并为`Context`资源，具有以下参数：

- `context`：`Context`资源的完全合格资源标识符。
- `artifacts`：`Artifact`资源的完全合格资源标识符列表。
- `executions`：`Execution`资源的完全合格资源标识符列表。

In [ ]:
clients["metadata"].add_context_artifacts_and_executions(
    context=context.name, artifacts=[artifact.name], executions=[execution.name]
)

查询上下文

您可以使用`query_context_lineage_subgraph()`方法查询`Context`资源的子图，方法参数如下：

- `context`：`Context`资源的完全合格的资源标识符。

In [ ]:
subgraph = clients["metadata"].query_context_lineage_subgraph(context=context.name)

print(subgraph)

### 删除`Artifact`资源

您可以使用`delete_artifact()`方法删除`Artifact`资源，参数如下：

- `name`：`Artifact`资源的完全限定资源标识符。

In [ ]:
clients["metadata"].delete_artifact(name=artifact.name)

### 删除`Execution`资源

您可以使用`delete_execution()`方法删除`Execution`资源，需要以下参数：

- `name`：`Execution`资源的完全合格资源标识符。

In [ ]:
clients["metadata"].delete_execution(name=execution.name)

### 删除 `Context` 资源

您可以使用 `delete_context()` 方法删除一个 `Context` 资源，需要以下参数：

- `name`：`Context` 资源的完全限定资源标识符。

In [ ]:
clients["metadata"].delete_context(name=context.name)

## 在`Vertex AI Pipeline`中跟踪ML元数据简介

Vertex AI Pipelines在执行管道时自动记录生成的指标和工件。然后，您可以使用SDK跟踪和分析管道运行中的指标和工件。

In [ ]:
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import (Artifact, Dataset, Input, Metrics, Model, Output,
                        OutputPath, component)

### 使用自定义组件创建一个包含3步的流水线

首先，您需要创建一个流水线来在`Vertex AI Pipelines`上运行，包含以下自定义组件：

* `get_dataframe`：从一个BigQuery表中检索数据并将其转换为一个pandas DataFrame。
* `sklearn_train`：使用pandas DataFrame来训练和导出一个scikit-learn模型，同时记录一些指标。
* `deploy_model`：部署导出的scikit-learn模型到一个`Vertex AI Endpoint`资源。

#### get_dataframe组件

该组件执行以下操作：

* 使用BigQuery客户端库创建对一个BigQuery表的引用
* 下载BigQuery表并将其转换为一个打乱顺序的pandas DataFrame
* 将DataFrame导出为一个CSV文件

#### sklearn_train组件

该组件执行以下操作：

* 将CSV文件导入为一个pandas DataFrame
* 将DataFrame分割为训练集和测试集
* 训练一个scikit-learn模型
* 记录模型的指标
* 将模型工件保存为一个本地的`model.joblib`文件

#### deploy_model组件

该组件执行以下操作：

* 将scikit-learn模型上传到一个`Vertex AI Model`资源
* 将模型部署到一个`Vertex AI Endpoint`资源。

In [ ]:
@component(
    packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow", "db-dtypes"],
    base_image="python:3.9",
    output_component_file="create_dataset.yaml",
)
def get_dataframe(bq_table: str, output_data_path: OutputPath("Dataset")):
    from google.cloud import bigquery

    bqclient = bigquery.Client()
    table = bigquery.TableReference.from_string(bq_table)
    rows = bqclient.list_rows(table)
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )
    dataframe = dataframe.sample(frac=1, random_state=2)
    dataframe.to_csv(output_data_path)


@component(
    packages_to_install=["scikit-learn", "pandas", "joblib"],
    base_image="python:3.9",
    output_component_file="beans_model_component.yaml",
)
def sklearn_train(
    dataset: Input[Dataset], metrics: Output[Metrics], model: Output[Model]
):
    import pandas as pd
    from joblib import dump
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier

    df = pd.read_csv(dataset.path)
    labels = df.pop("Class").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)

    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train, y_train)
    score = skmodel.score(x_test, y_test)
    print("accuracy is:", score)

    metrics.log_metric("accuracy", (score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))
    dump(skmodel, model.path + ".joblib")


@component(
    packages_to_install=["google-cloud-aiplatform"],
    base_image="python:3.9",
    output_component_file="beans_deploy_component.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model],
):
    from google.cloud import aiplatform

    aiplatform.init(project=project, location=region)

    deployed_model = aiplatform.Model.upload(
        display_name="beans-model-pipeline",
        artifact_uri=model.uri.replace("model", ""),
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest",
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")

    # Save data to the output params
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

### 构建并编译管道

接下来，构建管道：

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/3step"


@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline.
    name="mlmd-pipeline",
)
def my_pipeline(
    bq_table: str = "",
    output_data_path: str = "data.csv",
    project: str = PROJECT_ID,
    region: str = REGION,
):
    dataset_task = get_dataframe(bq_table)

    model_task = sklearn_train(dataset_task.output)

    deploy_model(model=model_task.outputs["model"], project=project, region=region)

### 编译和执行管道的两个运行

接下来，您编译管道，然后运行两个单独的管道实例。在第一个实例中，您使用数据集的小版本对模型进行训练，在第二个实例中，您使用数据集的大版本对模型进行训练。

In [ ]:
NOW = datetime.now().isoformat().replace(".", ":")[:-7]

compiler.Compiler().compile(
    pipeline_func=my_pipeline, package_path="mlmd_pipeline.json"
)

run1 = aiplatform.PipelineJob(
    display_name="mlmd-pipeline",
    template_path="mlmd_pipeline.json",
    job_id="mlmd-pipeline-small-unique",
    parameter_values={"bq_table": "sara-vertex-demos.beans_demo.small_dataset"},
    enable_caching=True,
)

run2 = aiplatform.PipelineJob(
    display_name="mlmd-pipeline",
    template_path="mlmd_pipeline.json",
    job_id="mlmd-pipeline-large-unique",
    parameter_values={"bq_table": "sara-vertex-demos.beans_demo.large_dataset"},
    enable_caching=True,
)

run1.run()
run2.run()

run1.delete()
run2.delete()

! rm -f mlmd_pipeline.json *.yaml

### 比较管道运行

现在您已经有了两次完成的管道运行，您可以比较这些运行。

您可以使用`get_pipeline_df()`方法来访问运行的元数据。这里的`mlmd-pipeline`参数是您给管道命名的名称：

**或者，有关在 Vertex AI 控制台中检查管道工件和元数据的指导，请参阅[此 codelab](https://codelabs.developers.google.com/vertex-mlmd-pipelines#5)。**

In [ ]:
df = aiplatform.get_pipeline_df(pipeline="mlmd-pipeline")
print(df)

### 可视化管道运行

接下来，您可以使用matplotlib创建自定义可视化图表，以查看模型准确度与用于训练的数据量之间的关系。

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df["metric.dataset_size"], df["metric.accuracy"], label="Accuracy")
plt.title("Accuracy and dataset size")
plt.legend(loc=4)
plt.show()

### 查询您的`Metadatastore`资源

最后，在调用`list_artifacts()`方法时，通过指定一个`filter`参数来查询您的`Metadatastore`资源。

In [ ]:
FILTER = f'create_time >= "{NOW}" AND state = LIVE'
artifact_req = {
    "parent": metadata_store_id,
    "filter": FILTER,
}

artifacts = clients["metadata"].list_artifacts(artifact_req)

for _artifact in artifacts:
    print(_artifact)
    clients["metadata"].delete_artifact(name=_artifact.name)

### 删除`MetadataStore`资源

您可以使用`delete_metadata_store()`方法删除`MetadataStore`资源，需要提供以下参数：

- `name`：`MetadataStore`资源的完全限定资源标识符。

In [ ]:
metadata_store_id = (
    f"projects/{PROJECT_ID}/locations/{REGION}/metadataStores/my-metadata-store-unique"
)

clients["metadata"].delete_metadata_store(name=metadata_store_id)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于指导教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以删除在此指导教程中创建的单个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI